In [1]:
import cv2
from data_loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from painting_detector import *
from evaluation_funcs import *
import os

In [2]:
imgs_in_path = "C:/Users/laila/Downloads/qsd2_w3/qsd2_w3"
images, imgs_names = DataLoader({"dataset": imgs_in_path}).load_images_from_folder(extension="jpg", return_names=True)

In [3]:
def compute_bg_removal(imgs_with_bg, imgs_names, output_dir):
    all_cropped=[]
    for idx, img in enumerate(imgs_with_bg):
        detector = PaintingDetector(img)
        # Detect and crop paintings
        output_mask, cropped_paintings = detector.detect_and_crop_paintings()
        all_cropped.append(cropped_paintings)
        # Save mask
        name, ext = imgs_names[idx].rsplit(".", 1)
        filename = f"{name}.png"
        cv2.imwrite(os.path.join(output_dir, filename), output_mask)
    return all_cropped
        
def evaluate_bg_removal(candidates, ground_truth):
    [TP, FP, FN, TN] = performance_accumulation_pixel(candidates, ground_truth)
    [pixel_precision, pixel_accuracy, pixel_specificity, pixel_sensitivity] = performance_evaluation_pixel(TP, FP, FN, TN)
    return pixel_precision, pixel_sensitivity

In [4]:

masks_gt, masks_gt_names = DataLoader({"dataset": imgs_in_path}).load_images_from_folder(extension="png", return_names=True)
# Create output directory
output_path = f"{imgs_in_path}/output"
os.makedirs(output_path, exist_ok=True)
cropped_paintings = compute_bg_removal(images, imgs_names, output_path)




In [5]:
import pickle

# Dump the list into a pickle file
with open('cropped_paintings.pickle', 'wb') as f:
    pickle.dump(cropped_paintings, f)

In [6]:
# TASK 3 -------------------------------------------------------------------------
masks_without_bg, masks_names = DataLoader({"dataset": output_path}).load_images_from_folder(extension="png", return_names=True)
    
precision = []
recall = []
for idx, mask in enumerate(masks_without_bg):
    p, r = evaluate_bg_removal(mask, masks_gt[idx])
    precision.append(p)
    recall.append(r)
f1 = calculate_f1_score(precision, recall)
print("TASK 3: Compare masks ---------------")
print(f"Precision: {np.mean(precision)}")
print(f"Recall: {np.mean(recall)}")
print(f"F1: {f1}")
print("-------------------------------------")

TASK 3: Compare masks ---------------
Precision: 0.9871743837620577
Recall: 0.9238775230013773
F1: 0.954477710639598
-------------------------------------
